In [197]:
import pandas as pd 
import numpy as np 


In [198]:
data = pd.read_csv(r"D:\Faisal\Projects\DoorDash\dataset\transformed_data.csv")
data

,log__total_items,log__subtotal,log__num_distinct_items,log__total_onshift_dashers,log__total_busy_dashers,log__total_outstanding_orders,log__estimated_order_place_duration,log__estimated_store_to_consumer_driving_duration,log__time,remainder__market_id,remainder__store_id,remainder__order_protocol,remainder__non_null_primary_category,y
0,-0.397105,0.060429,-0.226018,1.771130,1.864183,1.879046,-0.611525,0.011760,-0.610146,4.0,4367,2.0,salad,4626.0
1,1.059156,1.333535,0.997950,-0.130303,-0.092059,-0.090882,1.520654,-0.942984,1.273931,2.0,4976,1.0,sandwich,3269.0
2,0.266858,-0.833721,-0.226018,-0.802265,-0.722374,-0.677123,1.520654,-2.039182,1.311483,6.0,6268,1.0,breakfast,2022.0
3,-1.504158,-1.176881,-1.394317,-1.237975,-1.143116,-0.492981,1.520654,1.020360,-0.087514,1.0,5234,1.0,sandwich,4812.0
4,1.710643,-0.108714,0.997950,-1.947308,-1.806808,-1.976658,1.520654,0.235641,1.192800,3.0,5135,4.0,middle-eastern,5138.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138311,-0.397105,-0.785102,-0.226018,-0.856352,-1.216351,-1.179298,-0.611525,0.425064,0.092099,1.0,914,3.0,pizza,2815.0
138312,0.266858,-0.432779,0.494195,-1.485958,-1.469352,-1.485907,1.520654,-1.527911,1.234437,2.0,5577,3.0,salad,2063.0
138313,-1.504158,-0.462970,-1.394317,0.474650,0.530465,0.968199,-0.611525,-1.002925,-0.610146,4.0,693,3.0,italian,4672.0
138314,0.266858,0.167368,0.494195,0.855425,0.839513,0.681142,-0.611525,-0.402456,-1.046164,2.0,601,3.0,mexican,1501.0


In [199]:
from sklearn.ensemble import RandomForestRegressor
from category_encoders import BaseNEncoder 
from sklearn.preprocessing import OneHotEncoder, LabelEncoder,PowerTransformer,TargetEncoder,StandardScaler
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error , r2_score 


In [200]:
from sklearn.metrics import r2_score ,mean_squared_error

In [201]:
X = data.drop("y",axis=1)
y = data["y"]

In [202]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [203]:
cat_features = ['remainder__order_protocol','remainder__market_id']
cat_imputer = Pipeline(steps=[
    ("one_hot", OneHotEncoder(handle_unknown='ignore',sparse_output=False)) 
])

power_features = ['remainder__store_id']
power_imputer = Pipeline(steps=[
    ("log" , PowerTransformer(method='yeo-johnson'))
])

target_features = ['remainder__non_null_primary_category']
target_imputer = Pipeline(steps=[
    ("target", TargetEncoder(cv=5,smooth=500,target_type='continuous')),
    ("power", StandardScaler())
])

CT = ColumnTransformer(transformers=[
    ("target", target_imputer, target_features),
    ("one_hot", cat_imputer, cat_features),
    ("log", power_imputer, power_features)

],remainder='passthrough')


transformer_2 = Pipeline(steps=[("transform", CT)])
transformer_2.set_output(transform='pandas')

transformed_X_train = transformer_2.fit_transform(X_train, y_train)
transformed_X_test = transformer_2.transform(X_test)
# transformed_X_train

In [204]:
import xgboost 
from xgboost import XGBRegressor

In [205]:
def metrix(model , X_test, y_test):
    d = {}
    y_pred =  model.predict(X_test)
    mse = mean_squared_error(y_test,y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test,y_pred)
    d["r2"] = r2 
    d["rmse"] = rmse
    return d


In [206]:
param_xg = {

        "device" : ["cuda"],
        "booster" : ["gbtree"],
        "n_estimators":np.arange(300,510,100),
        "num_parallel_tree":[10],
    #   "learning_rate":[0.1],
        "max_depth":[5,6,7],
        "subsample":[0.7],
        "gamma":[0.1,0.5,1]
}

In [207]:
xg_gr = GridSearchCV(XGBRegressor(),
                     param_grid=param_xg,
                     cv=2,
                     verbose=1)

xg_gr.fit(transformed_X_train,y_train)

Fitting 2 folds for each of 27 candidates, totalling 54 fits


GridSearchCV(cv=2,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'booster': ['gbtree'], 'device': ['cuda'],
                         'gamma': [0.1, 0.5, 1], 'max_depth': [5, 6, 7],
                         'n_estimators': array([300, 400, 500]),
                         'num_parallel_tree': [10], 'subsample': [0.7]},
             verbose=1)

In [208]:
xg_gr.best_params_

{'booster': 'gbtree',
 'device': 'cuda',
 'gamma': 0.1,
 'max_depth': 5,
 'n_estimators': 300,
 'num_parallel_tree': 10,
 'subsample': 0.7}

In [209]:
xg_gr.score(transformed_X_train,y_train)

0.5204510225427714

In [210]:
xg_gr.score(transformed_X_test,y_test)

0.36055557187745313

In [211]:
metrix(xg_gr,transformed_X_test,y_test)

{'r2': 0.36055557187745313, 'rmse': 870.2929247618787}

# RandomForestRegressor


In [8]:
rf = RandomForestRegressor(criterion="squared_error",max_features="sqrt")
rf.fit(transformed_X_train,y_train)

RandomForestRegressor(max_features='sqrt')

In [9]:
rf.score(transformed_X_test,y_test)

0.3042984204200896

In [18]:
param = {
    "n_estimators" : [100],
    "criterion" : ["squared_error"],
    "max_features" : ["sqrt"]
}

In [10]:
from sklearn.model_selection import RandomizedSearchCV

In [20]:
rf_rr = RandomizedSearchCV(rf,param_distributions=param,cv=2,verbose=1)
rf_rr.fit(transformed_X_train,y_train)

c:\Users\solat\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 2 folds for each of 1 candidates, totalling 2 fits


RandomizedSearchCV(cv=2, estimator=RandomForestRegressor(max_features='sqrt'),
                   param_distributions={'criterion': ['squared_error'],
                                        'max_features': ['sqrt'],
                                        'n_estimators': [100]},
                   verbose=1)

In [21]:
rf_rr.score(transformed_X_test,y_test)

0.30288875144880956

In [23]:
y_pred = rf_rr.predict(transformed_X_test)

In [24]:
r2 = r2_score(y_test,y_pred)
r2

0.30288875144880956

In [26]:
mse = mean_squared_error(y_test,y_pred)
print(mse)
rmse = np.sqrt(mse)
print(rmse)

825715.0279487041
908.6886309119885


In [39]:
def metrix(model , X_test, y_test):
    d = {}
    y_pred =  model.predict(X_test)
    mse = mean_squared_error(y_test,y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test,y_pred)
    d["r2"] = r2 
    d["rmse"] = rmse
    return d


In [28]:
param = {
    "n_estimators" : [100,125],
    "criterion" : ["squared_error"],
    "max_features" : ["sqrt"]
}

In [29]:
rf_rr2 = RandomizedSearchCV(RandomForestRegressor(),param_distributions=param,cv=3)
rf_rr2.fit(transformed_X_train,y_train)

c:\Users\solat\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(),
                   param_distributions={'criterion': ['squared_error'],
                                        'max_features': ['sqrt'],
                                        'n_estimators': [100, 125]})

In [30]:
rf_rr2.score(transformed_X_test,y_test)

0.3037906423103277

In [34]:
metrix(rf_rr2,transformed_X_test,y_test)

{'r2': 0.3037906423103277, 'rmse': 908.1006306353153}

# Decision Tree

In [11]:
from sklearn.tree import DecisionTreeRegressor 


In [12]:
dt = DecisionTreeRegressor(max_features="sqrt")
dt.fit(transformed_X_train,y_train)

DecisionTreeRegressor(max_features='sqrt')

In [13]:
dt.score(transformed_X_test,y_test)

-0.5092391253361646

In [14]:
dt.score(transformed_X_train,y_train)

0.9999995776904853

In [ ]:
params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
}

In [79]:
param_dt = {
    "max_features" : [None],
    "splitter" : ["best"],
    # 'max_depth': [4],
    # 'min_samples_leaf': [5,20],
    "criterion" : ["squared_error", "friedman_mse"]
}

In [75]:
dt_gr = RandomizedSearchCV(DecisionTreeRegressor(),
                           param_distributions=param_dt,
                           cv=3,verbose=1)

dt_gr.fit(transformed_X_train,y_train)

c:\Users\solat\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 4 candidates, totalling 12 fits


RandomizedSearchCV(cv=3, estimator=DecisionTreeRegressor(),
                   param_distributions={'criterion': ['squared_error',
                                                      'friedman_mse'],
                                        'max_depth': [4],
                                        'max_features': [None],
                                        'min_samples_leaf': [5, 20],
                                        'splitter': ['best']},
                   verbose=1)

In [76]:
dt_gr.best_params_


{'splitter': 'best',
 'min_samples_leaf': 5,
 'max_features': None,
 'max_depth': 4,
 'criterion': 'squared_error'}

In [77]:
dt_gr.score(transformed_X_test,y_test)

0.1288836566313002

In [78]:
dt_gr.score(transformed_X_train,y_train)

0.11557373748184152

In [68]:
metrix(dt_gr,transformed_X_test,y_test)

{'r2': 0.1460537230193215, 'rmse': 1005.7251806867807}

# GridSearchCV

In [116]:
param_dt = {
    "max_features" : [None],
    "splitter" : ["best"],
    'max_depth': [None,5],
    'min_samples_leaf': [1],
    "criterion" : ["squared_error", "friedman_mse"]
}

In [117]:
dt_grid = GridSearchCV(DecisionTreeRegressor(),
                       param_grid=param_dt,cv=2,verbose=1)

dt_grid.fit(transformed_X_train,y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


GridSearchCV(cv=2, estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['squared_error', 'friedman_mse'],
                         'max_depth': [None, 5], 'max_features': [None],
                         'min_samples_leaf': [1], 'splitter': ['best']},
             verbose=1)

In [118]:
dt_grid.best_params_

{'criterion': 'squared_error',
 'max_depth': 5,
 'max_features': None,
 'min_samples_leaf': 1,
 'splitter': 'best'}

In [119]:
dt_grid.score(transformed_X_test,y_test)

0.1460537230193215

In [120]:
dt_grid.score(transformed_X_train,y_train)

0.14294100181495106

In [95]:
metrix(dt_grid,transformed_X_test,y_test)

{'r2': 0.14967864616307514, 'rmse': 1003.5883052734437}

# Linear Regression

In [11]:
from sklearn.linear_model import LinearRegression 

In [39]:
lr = LinearRegression(fit_intercept=True)
lr.fit(transformed_X_train,y_train)

LinearRegression()

In [40]:
lr.score(transformed_X_test,y_test)

0.27068230790266723

In [41]:
lr.score(transformed_X_train,y_train)

0.2357641019238279

In [19]:
transformed_X_train.head()

,target__remainder__non_null_primary_category,one_hot__remainder__order_protocol_1.0,one_hot__remainder__order_protocol_2.0,one_hot__remainder__order_protocol_3.0,one_hot__remainder__order_protocol_4.0,one_hot__remainder__order_protocol_5.0,one_hot__remainder__order_protocol_6.0,one_hot__remainder__order_protocol_7.0,one_hot__remainder__market_id_1.0,one_hot__remainder__market_id_2.0,...,log__remainder__store_id,remainder__log__total_items,remainder__log__subtotal,remainder__log__num_distinct_items,remainder__log__total_onshift_dashers,remainder__log__total_busy_dashers,remainder__log__total_outstanding_orders,remainder__log__estimated_order_place_duration,remainder__log__estimated_store_to_consumer_driving_duration,remainder__log__time
103848,0.042204,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.702897,-0.397105,-0.342641,-0.226018,0.399316,0.478796,0.499111,-0.611525,0.817502,1.234437
53376,1.331419,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.327425,0.721996,1.269365,0.494195,-0.349817,0.315663,-0.563125,1.520654,0.990942,-1.046164
32820,-1.397063,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.459672,-1.504158,-1.204685,-1.394317,-0.389470,0.315663,-0.333916,-0.611525,0.119733,1.192800
73037,0.168865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.635393,1.534343,0.542850,0.997950,0.239187,0.315663,0.310210,-0.611525,0.834511,0.092099
72893,0.985375,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.221934,-0.397105,0.221458,-0.226018,1.213678,1.302747,1.444638,-0.611525,0.372476,-1.046164


In [27]:
y_train

103848    3750.0
53376     3517.0
32820     2161.0
73037     2223.0
72893     2785.0
           ...  
110268    2751.0
119879    5070.0
103694    2508.0
131932    2777.0
121958    1839.0
Name: y, Length: 110652, dtype: float64

In [20]:
d = transformed_X_train.to_numpy()
full_data = np.c_[d,y_train]

In [29]:
full_data.shape


(110652, 25)

In [22]:
X_train1 = full_data[:,:-1]
y_train1 = full_data[:,-1]

In [25]:
X_train1[:1]

array([[ 0.04220432,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.70289697,
        -0.39710548, -0.34264057, -0.2260182 ,  0.3993157 ,  0.47879585,
         0.49911115, -0.61152481,  0.81750223,  1.23443697]])

In [26]:
y_train1

array([3750., 3517., 2161., ..., 2508., 2777., 1839.])

In [33]:
data_percent = len(X_train1) * 0.2
data_percent = int(data_percent)
data_percent

22130

In [34]:
X_train_percent = X_train1[:data_percent,:]
y_train_percent = y_train1[:data_percent]

In [35]:
X_train_percent.shape

(22130, 24)

# SVM 

In [121]:
from sklearn.svm import SVR 

In [122]:
sv = SVR()

In [123]:
sv.fit(transformed_X_train,y_train)

SVR()

In [124]:
sv.score(transformed_X_test,y_test)

0.1401101816135455

# SVM Hyperparameter 

In [130]:
sv1 = SVR(kernel="sigmoid",degree=5)
sv1.fit(transformed_X_train,y_train)

SVR(degree=5, kernel='sigmoid')

In [131]:
sv1.score(transformed_X_test,y_test)

0.14785342383884803

In [132]:
sv1.score(transformed_X_train,y_train)

0.12407833970214055

In [133]:
! nvidia-smi

Tue Mar  5 16:01:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.34                 Driver Version: 537.34       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8               4W /  50W |    108MiB /  4096MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [136]:
! python -m pip install GPUtil

DEPRECATION: Loading egg at c:\users\solat\appdata\local\programs\python\python311\lib\site-packages\ml_project-0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7400 sha256=052bd7c0112c0084369617b1c91854c3999a75e19970c536ea9b54145991774a
  Stored in directory: c:\users\solat\appdata\local\pip\cache\wheels\2b\4d\8f\55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil


In [137]:
import GPUtil 


In [139]:
GPUtil.getAvailable()

[0]

In [140]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  5% |  3% |


In [144]:
GPUtil.getGPUs()


# Phase 2 Transformation

In [2]:
import pandas as pd 
import numpy as np 

In [3]:
phase_2  = pd.read_csv(r"D:\Faisal\Projects\DoorDash\dataset\final_doordash.csv")
f = phase_2.copy()

In [4]:
f.isna().sum()

market_id                                       0
store_id                                        0
order_protocol                                  0
total_items                                     0
subtotal                                        0
num_distinct_items                              0
total_onshift_dashers                           0
total_busy_dashers                              0
total_outstanding_orders                        0
estimated_order_place_duration                  0
estimated_store_to_consumer_driving_duration    0
delivery_time                                   0
non_null_primary_category                       0
time                                            0
dtype: int64

In [5]:
f.head()

,market_id,store_id,order_protocol,total_items,subtotal,num_distinct_items,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,delivery_time,non_null_primary_category,time
0,1.0,1845,1.0,4,3441,4,33.0,14.0,21.0,446,861.0,3779.0,american,22
1,2.0,5477,2.0,1,1900,1,1.0,2.0,2.0,446,690.0,4024.0,indian,21
2,2.0,5477,3.0,4,4771,3,8.0,6.0,18.0,446,289.0,1586.0,indian,0
3,1.0,2841,1.0,1,1525,1,5.0,6.0,8.0,446,795.0,2273.0,italian,3
4,1.0,2841,1.0,2,3620,2,5.0,5.0,7.0,446,205.0,2988.0,italian,2


In [6]:
cat_vals = f["non_null_primary_category"].value_counts()
cat_vals

non_null_primary_category
american             18038
pizza                15564
mexican              15491
burger                9744
sandwich              8909
                     ...  
russian                 10
african                 10
belgian                  2
chocolate                1
alcohol-plus-food        1
Name: count, Length: 73, dtype: int64

In [7]:
cat_vals.values

array([18038, 15564, 15491,  9744,  8909,  8211,  8057,  7965,  6575,
        6298,  6291,  6181,  5442,  5054,  4686,  3672,  3511,  2971,
        2645,  2561,  2192,  1941,  1937,  1748,  1569,  1500,  1455,
        1426,  1413,  1022,   990,   750,   634,   631,   574,   550,
         490,   436,   313,   301,   297,   293,   267,   238,   236,
         230,   210,   183,   138,   123,   122,   110,    98,    93,
          68,    64,    58,    53,    52,    35,    28,    26,    24,
          23,    21,    20,    12,    12,    10,    10,     2,     1,
           1], dtype=int64)

In [8]:
top_30 = cat_vals[:30].index
top_30

Index(['american', 'pizza', 'mexican', 'burger', 'sandwich', 'chinese',
       'dessert', 'japanese', 'italian', 'fast', 'indian', 'thai',
       'vietnamese', 'mediterranean', 'breakfast', 'other', 'salad', 'greek',
       'barbecue', 'seafood', 'asian', 'sushi', 'cafe', 'alcohol', 'korean',
       'catering', 'smoothie', 'middle-eastern', 'hawaiian', 'steak'],
      dtype='object', name='non_null_primary_category')

In [9]:
cat_vals2 = cat_vals.replace(cat_vals.values, [i if i in top_30 else "others" for i in cat_vals.index])
cat_vals2

non_null_primary_category
american             american
pizza                   pizza
mexican               mexican
burger                 burger
sandwich             sandwich
                       ...   
russian                others
african                others
belgian                others
chocolate              others
alcohol-plus-food      others
Name: count, Length: 73, dtype: object

In [10]:
f["cuisine_category"] = f["non_null_primary_category"].map(cat_vals2)
f.head()

,market_id,store_id,order_protocol,total_items,subtotal,num_distinct_items,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,delivery_time,non_null_primary_category,time,cuisine_category
0,1.0,1845,1.0,4,3441,4,33.0,14.0,21.0,446,861.0,3779.0,american,22,american
1,2.0,5477,2.0,1,1900,1,1.0,2.0,2.0,446,690.0,4024.0,indian,21,indian
2,2.0,5477,3.0,4,4771,3,8.0,6.0,18.0,446,289.0,1586.0,indian,0,indian
3,1.0,2841,1.0,1,1525,1,5.0,6.0,8.0,446,795.0,2273.0,italian,3,italian
4,1.0,2841,1.0,2,3620,2,5.0,5.0,7.0,446,205.0,2988.0,italian,2,italian


In [11]:
f["cuisine_category"].nunique()

31

In [12]:
f.drop("non_null_primary_category",axis=1,inplace=True)

# Encoding 

In [139]:
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer 
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error,r2_score 


In [14]:
f.head()

,market_id,store_id,order_protocol,total_items,subtotal,num_distinct_items,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,delivery_time,time,cuisine_category
0,1.0,1845,1.0,4,3441,4,33.0,14.0,21.0,446,861.0,3779.0,22,american
1,2.0,5477,2.0,1,1900,1,1.0,2.0,2.0,446,690.0,4024.0,21,indian
2,2.0,5477,3.0,4,4771,3,8.0,6.0,18.0,446,289.0,1586.0,0,indian
3,1.0,2841,1.0,1,1525,1,5.0,6.0,8.0,446,795.0,2273.0,3,italian
4,1.0,2841,1.0,2,3620,2,5.0,5.0,7.0,446,205.0,2988.0,2,italian


In [15]:
f.columns

Index(['market_id', 'store_id', 'order_protocol', 'total_items', 'subtotal',
       'num_distinct_items', 'total_onshift_dashers', 'total_busy_dashers',
       'total_outstanding_orders', 'estimated_order_place_duration',
       'estimated_store_to_consumer_driving_duration', 'delivery_time', 'time',
       'cuisine_category'],
      dtype='object')

In [16]:
X = f.drop("delivery_time",axis=1)
y = f["delivery_time"]


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [18]:
cuisine_feature = ["cuisine_category","market_id","order_protocol"]

cuisine_imputer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="error"))
])

CT = ColumnTransformer(transformers=[
    ("o", cuisine_imputer,cuisine_feature)
],remainder="passthrough")

transformer = Pipeline(steps=[("CT", CT)])

trans_train_X = transformer.fit_transform(X_train)
trans_test_X = transformer.transform(X_test)


In [19]:
trans_train_X.shape

(138316, 54)

In [20]:
r = LinearRegression()
r.fit(trans_train_X,y_train)


LinearRegression()

In [21]:
r.score(trans_train_X, y_train)

0.23727123283818619

In [22]:
r.score(trans_test_X,y_test)

0.23080632274237112

In [23]:
dt2 = DecisionTreeRegressor(criterion ='squared_error', 
                            max_features=None,
                            max_depth=10,
                            splitter="best",
                            min_samples_split=3)
dt2.fit(trans_train_X,y_train)

DecisionTreeRegressor(max_depth=10, min_samples_split=3)

In [24]:
dt2.score(trans_train_X,y_train)

0.2929804778630264

In [25]:
dt2.score(trans_test_X,y_test)

0.17358374060148918

In [26]:
param_dt = {
    "max_features" : [None],
    "splitter" : ["best"],
    'max_depth': [9,10,11],
    'min_samples_leaf': [1,3,5],
    "criterion" : ["squared_error"]
}

In [27]:
r_gr = GridSearchCV(DecisionTreeRegressor(),
                    param_grid=param_dt,cv=3,verbose=1)

r_gr.fit(trans_train_X,y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


GridSearchCV(cv=3, estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['squared_error'],
                         'max_depth': [9, 10, 11], 'max_features': [None],
                         'min_samples_leaf': [1, 3, 5], 'splitter': ['best']},
             verbose=1)

In [28]:
r_gr.best_params_

{'criterion': 'squared_error',
 'max_depth': 10,
 'max_features': None,
 'min_samples_leaf': 5,
 'splitter': 'best'}

In [29]:
r_gr.score(trans_train_X,y_train)

0.2612461373834388

In [30]:
r_gr.score(trans_test_X,y_test)

0.1853710797002368

# SVM

In [31]:
# from sklearn.svm import SVR 
# sv = SVR()

In [32]:
# sv.fit(trans_train_X,y_train)

In [33]:
# sv.score(trans_train_X,y_train)

In [34]:
# sv.score(trans_test_X,y_test)

# Ensemble Learning

In [156]:
randomf = RandomForestRegressor(max_features="sqrt",n_estimators= 100,)

In [157]:
randomf.fit(trans_train_X,y_train)

RandomForestRegressor(max_features='sqrt')

In [158]:
randomf.score(trans_train_X,y_train)

0.90036017974483

In [159]:
randomf.score(trans_test_X,y_test)

0.2741876612074359

In [160]:
metrix(randomf,trans_test_X,y_test)

{'r2': 0.2741876612074359, 'rmse': 995.1511394132311}

# XGBoost

In [41]:
# ! python -m pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/24/ec/ad387100fa3cc2b9b81af0829b5ecfe75ec5bb19dd7c19d4fea06fb81802/xgboost-2.0.3-py3-none-win_amd64.whl.metadata
Using cached xgboost-2.0.3-py3-none-win_amd64.whl (99.8 MB)


DEPRECATION: Loading egg at c:\users\solat\appdata\local\programs\python\python311\lib\site-packages\ml_project-0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
import xgboost

In [43]:
from xgboost import XGBRegressor

In [162]:
xr = XGBRegressor(device="cuda",booster="gbtree",
                  n_estimators=500,
                  num_parallel_tree=20,
                #   learning_rate=0.1,
                  max_depth=6,
                  subsample=0.8,
                  gamma=1)
xr.fit(trans_train_X,y_train)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=20, random_state=None, ...)

In [163]:
xr.score(trans_train_X,y_train)

0.6706838669944777

In [164]:
xr.score(trans_test_X,y_test)

0.3233865919230843

In [165]:
def metrix(model , X_test, y_test):
    d = {}
    y_pred =  model.predict(X_test)
    mse = mean_squared_error(y_test,y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test,y_pred)
    d["r2"] = r2 
    d["rmse"] = rmse
    return d


In [146]:
metrix(xr,trans_test_X,y_test)

{'r2': 0.3294143220552733, 'rmse': 956.5419210277038}

In [181]:
param_xg = {

        "device" : ["cuda"],
        "booster" : ["gbtree"],
        "n_estimators":np.arange(300,510,100),
        "num_parallel_tree":[10],
    #   "learning_rate":[0.1],
        "max_depth":[5,6,7],
        "subsample":[0.7],
        "gamma":[0.1,0.5,1]
}

In [182]:
xg_gr = GridSearchCV(XGBRegressor(),
                     param_grid=param_xg,
                     cv=2,
                     verbose=1)

xg_gr.fit(trans_train_X,y_train)

Fitting 2 folds for each of 27 candidates, totalling 54 fits


GridSearchCV(cv=2,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'booster': ['gbtree'], 'device': ['cuda'],
                         'gamma': [0.1, 0.5, 1], 'max_depth': [5, 6, 7],
                         'n_estimators': array([300, 400, 500]),
                         'num_parallel_tree': [10], 'subsample': [0.7]},
             verbose=1)

In [183]:
xg_gr.best_params_

{'booster': 'gbtree',
 'device': 'cuda',
 'gamma': 0.1,
 'max_depth': 5,
 'n_estimators': 300,
 'num_parallel_tree': 10,
 'subsample': 0.7}

In [184]:
xg_gr.score(trans_train_X,y_train)

0.5046134026101246

In [185]:
xg_gr.score(trans_test_X,y_test)

0.3248232671231148

In [188]:
metrix(xg_gr,trans_test_X,y_test)

{'r2': 0.3248232671231148, 'rmse': 959.8107394542167}